# KZG

The construction of the "oracles" will be solved using the concept of "polynomial commitments". These are values that are binded to the polynomial without revealing its exact form. This value is derived from the polynomial in such a way that it’s very hard to figure out the original polynomial just by looking at the commitment. However, this commitment is still uniquely tied to the original polynomial (with overwhelming probability).

In KZG these will be instantiated using two ideas: (Elliptic Curve) Discrete Log Problem + Schwarz-Zippel Lemma. Say I have a polynomial $f$ and compute $f(\tau) G$ for some random element $\tau$. Since the discrete log problem is hard in elliptic curves, it's very unlikely that someone with access to $f(\tau)G$ will be able to obtain $f(\tau)$. So, the value $f(\tau) G$ reveals nothing about the original polynomial $f$. On the other hand, if $g(\tau) G$ equals $f(\tau) G$ for some other polynomial $g$, then $f(\tau)$ equals $g(\tau)$. Since $\tau$ is random, we deduce that $f$ equals $g$ as polynomials. And thus, $f(\tau) G$ is uniquely tied to the original polynomial with overwhelming probability.

One first problem is that the above works as long as the random value $\tau$ is not known to the committing party. Or otherwise it could craft polynomials that are different but take the same value at $\tau$, breaking the purpose of the commitment. So, how can anyone compute $f(\tau)G$ without knowing $\tau$? The solution is what's called an SRS. There are protocols that can be run between multiple parties producing the following list of points

$$G, \tau G, \tau^2 G, \dots, \tau^N G,$$

for a random element $\tau$ not known to any party (as long as one party is honest). This list of elements is called an SRS.
Having such list solves the problem, since:
- The value of $\tau$ can't be computed off the list assuming the DLOG problem is hard.
- The commiting party can compute $f(\tau) G$ as $$f_0G + f_1 \tau G + f_2 \tau^2 G + \cdots + f_k \tau^k G,$$ where $f_i$ is the $i$-th coefficient of $f$ in the monomial basis. The only constraint is that the degree of $f$ must be at most $N$.


## The KZG Polynomial Commitment Scheme

Let $G$ be a point in an elliptic curve with order $p$. If $x \in \mathbb{F}_p$, then $x G$ is defined as $G + \cdots G$ $x$ times.

**Commitment for $f \in \mathbb{F}_p[X]$**: return $$f(\tau)G$$

**Opening for $z \in \mathbb{F}_p$**: Compute $t = (p - p(z)) / (X - z)$, return $p(z)$ and $t(\tau) G$

**Verify**: return Accept if $e( f(\tau) G - y G, G)$ equals $e(t(\tau), \tau G - y G)$

In [9]:
from typing import List
from zk_adventures_types import F, Polynomial, CURVE_GENERATOR as G, CURVE_NEUTRAL_ELEMENT as O, pairing, E
from kzg_srs import SRS

In [10]:
def commit_polynomial_for_srs(p: Polynomial, srs):
    """Returns the commitment of `p` using the given SRS"""
    result = O
    p_coeff = p.list()
    for i in range(len(p_coeff)):
        result += p_coeff[i] * srs[i]
    return result

In [11]:
#[TEST]

X = Polynomial.monomial(1)

polynomial = 47077*X^6 + 4097*X^5 + 64206*X^4 + 38157*X^3 + 4872*X^2 + 10925*X + 48147
expected_commitment = E(107534, 213309, 1)
assert commit_polynomial_for_srs(polynomial, SRS) == expected_commitment

In [12]:
def open_polynomial_for_srs(p: Polynomial, z, srs):
    """Returns `p(z)` and the KZG opening proof of it using the given SRS"""
    t = (p - p(z)) // (X - z)
    return p(z), commit_polynomial_for_srs(t, srs)

In [13]:
#[TEST]

y, proof = open_polynomial_for_srs(polynomial, 0xcafecafe, SRS)
assert y == 48147
assert proof == E(40401, 209607, 1)

In [14]:
class KZGProver:
    def __init__(self, srs):
        self._srs = srs
        
    def commit_polynomial(self, p: Polynomial):
        return commit_polynomial_for_srs(p, self._srs)
    def open_polynomial(self, p: Polynomial, z):
        return open_polynomial_for_srs(p, z, self._srs)

class KZGVerifier:
    def __init__(self, srs):
        self._G = srs[0]
        self._tau_G = srs[1]
    
    def verify(self, commitment, z, y, proof):

        yg = y * self._G
        zg = z * self._G
        
        pairing_left = pairing(
            commitment - yg,
            self._G
        )

        pairing_right = pairing(
            proof,
            self._tau_G - zg
        )
        return pairing_left == pairing_right

In [15]:
#[TEST]

prover = KZGProver(SRS)

p = Polynomial.random_element(21)
commitment = prover.commit_polynomial(p)
z = F.random_element()
y, proof = prover.open_polynomial(p, z)

verifier = KZGVerifier(SRS)
assert verifier.verify(commitment, z, y, proof)

## Batch KZG: Many polynomials on single point

In Plonk it is common to have multiple polynomials $f_1,\dots,f_k$ that need to be commited and later opened at the same point $z$. The naive approach is to do individual commitments and openings for every polynomial. Each opening involves computing two pairings, and this is costly. There's an optimization to commit to a large number of polynomials but later on only computing two pairings to open all of them at the same value $z$.
For this to work both the KZG prover and KZG verifier will need to have access to a random value $\alpha$ that we assume in this exercise to be given truly random. In practice it chosen given by the Plonk verifier.

There's a theorem backing up this idea. Let $z$ be a field element and $f_0, \dots, f_k$ polynomials. Suppose one of the polynomials, say $f_i$, is not divisible by $X - z$. Then, with high probability ($1 - k/p$) the polynomial $$f_0 + \alpha f_1 + \cdots + \alpha^k f_k$$
is not divisible by $X - z$.

So, it is safe to work directly with the polynomial $\sum_{i=0}^k\alpha^if_i$ to start with and do a single commit and single open over it.

### Batch KZG

Let $\alpha$ be a random value (for example previously chosen by the Plonk verifier)

**Commitment for $(f_1, \dots, f_k) \in \mathbb{F}_p[X]^k$**: return $$\sum_{i=1}^k\alpha^if_i(\tau)G$$

**Opening for $z \in \mathbb{F}_p$**: Compute $$t = (\sum_{i=1}^k\alpha^if_i - \sum_{i=1}^k\alpha^if_i(z)) / (X - z)$$Return $\sum_{i=1}^k\alpha^if_i(z)$ and $t(\tau) G$

**Verify**: return Accept if $e( f(\tau) G - y G, G)$ equals $e(t(\tau), \tau G - y G)$, where $$f = f_0 + \alpha f_1 + \cdots \alpha^k f_k$$ and $$y = y_0 + \alpha y_1 + \cdots + \alpha^k y_k$$

In [28]:
def batch_commit_polynomials_for_srs(polynomials: List[Polynomial], alpha: F, srs):
    """Returns the commitment of the list of polynomials using the given random 
    challenge `alpha` and the given SRS"""
    comb_poly = 0
    for i,p in enumerate(polynomials):
        comb_poly += alpha ** i * p
    return commit_polynomial_for_srs(comb_poly, srs)
        
        

In [29]:
#[TEST]

polynomials = [
    51407*X^3 + 51302*X^2 + 49772*X + 61489,
    15476*X^3 + 32649*X^2 + 4888*X + 59566,
    43886*X^3 + 64965*X^2 + 60621*X + 33955
]

alpha = F(0xcafe)

expected_commitment = E(10903, 113294, 1)
assert batch_commit_polynomials_for_srs(polynomials, alpha, SRS) == expected_commitment

In [34]:
def batch_open_polynomials_for_srs(polynomials: List[Polynomial], z: F, alpha: F, srs):
    """Returns `[p_0(z), p_1(z), ..., p_k(z)]` and the KZG opening proof of them using the given SRS"""

    comb_poly = 0
    for i,p in enumerate(polynomials):
        comb_poly += alpha ** i * p
    _, proof = open_polynomial_for_srs(comb_poly, z, srs)
    return [p(z) for p in polynomials], proof
        

In [35]:
#[TEST]

alpha = F(0xcafe)
z = F(0xdeadbeef)
evaluations, proof = batch_open_polynomials_for_srs(polynomials, z, alpha, SRS)
assert evaluations == [10394, 36975, 45154]
assert proof == E(99056, 222835, 1)

In [42]:
class BatchKZGProver:
    def __init__(self, srs):
        self._srs = srs
        
    def commit_polynomials(self, polynomials: List[Polynomial], alpha: F):
        return batch_commit_polynomials_for_srs(polynomials, alpha, self._srs)
    
    def open_polynomials(self, polynomials: List[Polynomial], z: F, alpha: F):
        return batch_open_polynomials_for_srs(polynomials, z, alpha, self._srs)
    
class BatchKZGVerifier:
    def __init__(self, srs):
        self._G = srs[0]
        self._tau_G = srs[1]
    
    def verify(self, commitment, z, alpha, evaluations, proof):
        
        comb_yg = O
        for i,y in enumerate(evaluations):
            comb_yg += alpha ** i * y * self._G

        zg = z * self._G
        
        pairing_left = pairing(
            commitment - comb_yg,
            self._G
        )

        pairing_right = pairing(
            proof,
            self._tau_G - zg
        )
        return pairing_left == pairing_right

In [43]:
prover = BatchKZGProver(SRS)

polynomials = [Polynomial.random_element(15) for _ in range(10)]
alpha = F.random_element()
commitment = prover.commit_polynomials(polynomials, alpha)
z = F.random_element()
evaluations, proof = prover.open_polynomials(polynomials, z, alpha)

verifier = BatchKZGVerifier(SRS)
assert verifier.verify(commitment, z, alpha, evaluations, proof)